# Genetic Algorithm and Cellular Automata

## Genetic Algorithm

In [180]:
import ioh
import random
import numpy as np
from algorithm import Algorithm

class RandomSearch(Algorithm):
    '''An example of Random Search.'''

    def __call__(self, problem: ioh.problem.Integer) -> None:
        self.y_best: float = float("inf")
        for iteration in range(self.max_iterations):
            # Generate a random bit string
            x: list[int] = [random.randint(0, 1) for _ in range(problem.meta_data.n_variables)]
            # Call the problem in order to get the y value    
            y: float = problem(x)
            # update the current state
            self.y_best = max(self.y_best, y)


class GeneticAlgorithm(Algorithm):

    def __init__(self, pop_size=22, dim=10, seed=23, base=2, mutation_type="swap", mutation_rate=0.5, selection="proportional"):
        assert selection == "proportional" or selection == "rank", "selection only accepts \"proportional\" or \"rank\""
        super().__init__(max_iterations=50000)
        self.mutation_type = mutation_type
        self.dim = dim
        self.seed = seed
        self.base = base
        self.population_size = pop_size
        np.random.seed(self.seed)
        self.population = self.generate_population()
        self.y_best = 0.0
        self.x_best : list[int] = []
        self.fitness_scores = []
        self.mutation_rate = mutation_rate
        self.selection_method = selection


    def generate_candidate(self):
        """
        Generate a candidate i for the population self.population
        """
        candidate = np.random.randint(self.base, size=self.dim, dtype=int)
        return candidate


    def generate_population(self):
        """
        Generate a candidate for every i in self.population_size
        """
        return [self.generate_candidate() for i in range(self.population_size)]


    def print_pop_size(self):
        """
        Print the population size
        """
        print(self.population_size)

    def evaluate_population(self, problem):
        [self.fitness(self.population[i], problem) for i in range(self.population_size)]



    def selection(self, problem, k=2):
        """
        * Calculate fitness score/weight for every candidate
        * Randomly select k candidates given selection probabilities
        * return a list of two candidates
        """
        candidates = np.empty(shape=[0, 2])
        self.evaluate_population(problem)
        for candidate in range(len(self.population)):
            candidates = np.append(candidates, [[self.population[candidate], self.selection_probabilities(candidate)]], axis=0)
        weights = np.array(candidates[:,1], dtype=float)
        np_candidates = np.array(candidates[:,0])
        
        if self.selection_method=="rank":
            sorted_candidates = np.asarray(sorted(candidates, key=lambda l: l[1]))
            np_candidates = np.array(sorted_candidates[:,0])
            rank_weights = [] 
            for i in range(1, self.population_size+1):
                rank_weights.append(i)
            sum_rank_weights = sum(rank_weights)
            for i in range(self.population_size):
                rank_weights[i] = rank_weights[i]/sum_rank_weights
            selected = np.random.choice(np_candidates, p=rank_weights, size=2)
        else:
            selected = np.random.choice(np_candidates, p=weights, size=2)
        return selected


    def selection_probabilities(self, candidate_idx):
        """
        Return candidate fitness score / population fitness score
        """
        prob = (self.fitness_scores[candidate_idx] / self.population_fitness())
        return prob


    def mutation(self, offspring):
        """
        For every child in the offspring apply either a swap, reverse or an insert mutation
        """
        new_offspring = []
        prob = np.random.random_sample(self.population_size)
        idx = 0
        if self.mutation_type == "swap":
            for child in offspring:
                if prob[idx] > 1-self.mutation_rate:
                    child = self.swap_mutation(child)
                    new_offspring.append(child)
                else:
                    new_offspring.append(child)
                idx += 1
        elif self.mutation_type == "insert":
            for child in offspring:
                if prob[idx] > 1-self.mutation_rate:
                    child = self.insert_mutation(child)
                    new_offspring.append(child)
                else:
                    new_offspring.append(child)
                idx += 1
        else:
            for child in offspring:
                if prob[idx] > 1-self.mutation_rate:
                    child = self.reverse_mutation(child)
                    new_offspring.append(child)
                else:
                    new_offspring.append(child)
                idx += 1

        return new_offspring


    def swap_mutation(self, child):
        """
        Apply a swap mutation, by randomly selecting 2 genes and swapping them.
        """
        rand_idx = np.random.randint(self.dim,size=2)
        temp = child[rand_idx[1]]
        child[rand_idx[1]] = child[rand_idx[0]]
        child[rand_idx[0]] = temp
        return child



    def insert_mutation(self, child):
        """
        * Apply an insert mutation, by randomly selecting an i and a j in the genome.
        * Insert j at position i+1 and delete it's previous index
        """
        child = list(child)
        i_idx = np.random.randint(len(child))
        j_idx = np.random.randint(len(child))

        while i_idx == j_idx:
            i_idx = np.random.randint(len(child))
            j_idx = np.random.randint(len(child))
        child.insert(i_idx+1, child[j_idx])
        if j_idx > i_idx:
            child.pop(j_idx+1)
        else:
            child.pop(j_idx)
        return child

    def reverse_mutation(self, child):
        child = list(child)
        i_idx = np.random.randint(len(child))
        j_idx = np.random.randint(len(child))

        while i_idx == j_idx or i_idx > j_idx:
            i_idx = np.random.randint(len(child))
            j_idx = np.random.randint(len(child))
        sublist = child[i_idx:j_idx]
        sublist.reverse()
        child[i_idx:j_idx] = sublist
        return child

    def crossover(self, parents):
        """
        For every child in the offspring apply a crossover.
        """
        if(len(parents) != 2):
            raise ValueError("There should be 2 parents")
        offspring = []
        for i in range(self.population_size):
            offspring.append(self.crossover_operator(parents))

        return offspring


    def crossover_operator(self, parents):
        """
        * Given two parents, create a child based on a random uniform probability
        * Children are created by a simple treshold. If the probability of a certain index in idx_prob is higher than 0.5,
        * then a gene from parent A is given, else: parent B
        * This is done for n = self.population_size
        """
        parentA = np.array(parents[0], dtype=int)
        parentB = np.array(parents[1], dtype=int)
        child = np.empty(shape=[0, 1], dtype=int)
        idx_prob = np.random.random_sample(self.dim)
        for i in range(len(idx_prob)):
            if idx_prob[i] > 0.5:
                child = np.append(child, parentA[i])
            else:
                child = np.append(child, parentB[i])
        return child


    def fitness(self, candidate, problem):
        """
        Return the fitness metric
        """
        fitness = problem(candidate)
        self.fitness_scores.append(fitness)
        return fitness


    def population_fitness(self):
        """"
        Calculate the fitness score for the whole population
        Calculation of the fitness score is explained in fitness(self, candidate)
        """
        pop_fitness = sum(self.fitness_scores)
        return pop_fitness


    def __call__(self, problem: ioh.problem.Integer) -> None:
        self.dim = problem.meta_data.n_variables
        self.population = self.generate_population()
        # assert problem.state.evaluations > self.max_iterations
        # print('best solution', problem.state.current_best.x)
        # while (problem.state.evaluations < self.max_iterations) and (problem.state.current_best.x != problem.objective.x):
        while (problem.state.evaluations < self.max_iterations) and problem.state.current_best.x != 100.0:
            # print('test 3')
            parents = self.selection(problem=problem)
            offspring = self.crossover(parents)
            mutated_offspring = self.mutation(offspring)
            self.population = mutated_offspring
            for cand in range(len(self.population)):
                new_y = self.fitness_scores[cand]
                if new_y > self.y_best:
                    self.y_best = new_y
                    self.x_best = list(self.population[cand])
                    # print('best x: ', self.x_best, 'with y value of:', self.y_best)
            self.fitness_scores.clear()
        problem(self.x_best)
        # print('evaluations: ', problem.state.evaluations)
        # return problem.state.current_best
        pass






In [182]:
def main():

    # Set a random seed in order to get reproducible results
    random.seed(42)
    # np.random.seed(2)

    # Get a problem from the IOHexperimenter environment
    problem: ioh.problem.Integer = ioh.get_problem(2, 1, 10, "Integer")
    
    # print(problem([1,1,1,0,1,0]))
    # print(problem.objective.x)

    #Instantiate the algoritm, you should replace this with your GA implementation
    algorithm = GeneticAlgorithm(mutation_type="swap",selection="rankd",pop_size=20)

    # Run the algoritm on the problem
    algorithm(problem)

    # Inspect the results
    print("Best solution found:")
    print("".join(map(str, problem.state.current_best.x)))
    print("With an objective value of:", problem.state.current_best.y)
    print()
    
main()

AssertionError: selection only accepts "proportional" or "rank"

In [125]:
# problem: ioh.problem.Integer = ioh.get_problem(2, 1, 10, "Integer")

# problem.objective.x

## Cellular Automata and the inversing problem:

In [63]:
import typing
import math
import ioh

from implementation import RandomSearch

class CellularAutomata:
    '''Skeleton CA, you should implement this.'''
    
     
    def __init__(self, rule_number: int, neighborhood_radius = 1, length = 10, max_steps = 50, edge=0, base = 2):
        """
        TODO: 
        * Define a rule
        * Given a certain rule, such as 112 -> Convert it to binary
        * Most significant bit is left
        * checks implementeren om te kijken of arrays wel van goede lengte zijn
        * checks implementeren om te kijken of een rule wel valid is
        """
        # self.init_state = init_state
        self.time_step = 0
        self.state = []
        self.rule_number = rule_number
        self.radius = neighborhood_radius
        self.length = length
        self.max_steps = max_steps
        self.edge = edge
        self.base = base
        # self.curr_state = init_state
        self.possible_rules = int(math.pow(self.base,2*self.radius+1))
        
        pass
    
    
    def rule_to_base_k(self):
        """
        Convert a rule to a certain base-k representation,
        and then save it to a list
        """
        rule_converted = np.base_repr(self.rule_number, base=self.base)
        rule_list = [int(x) for x in str(rule_converted)]
        while(len(rule_list) != self.possible_rules):
            rule_list.insert(0,0)
        return rule_list
    
    
    def convert_to_binary(self, num, length):
        """
        Convert a base 10 number to a base 2 representation, given a certain length for the bitstring
        """
        bitstring = np.binary_repr(num, width=length)
        print('type of bitstring: ', type(bitstring))
        return bitstring
    
    def convert_to_other_base(self, num, length):
        """
        Convert a base 10 number to a base k representation, given a certain length for the output
        """
        converted_num = np.base_repr(num, base=self.base)
        # print('length of basestring:', length)
        while(len(converted_num) != length):
            converted_num_list = [int(x) for x in str(converted_num)]
            converted_num_list.insert(0,0)
            converted_str = ''
            converted_num = converted_str.join(map(str, converted_num_list))
        # print('len of converted bitstring:',len(converted_num))
        return converted_num
    
    def possible_neighborhood_states(self):
        """
        This function returns a list with the possible states, that will lead to a non-zero cells in the next time step.
        ie, if the list contains 101 for base 2, it means if for a certain cell C the neighborhood looks like 101 =>
        Cell C will be 1 in timestep t+1
        """
        evaluate_rule = self.rule_to_base_k()
        neighborhood_states = []
        neighborhood_state = self.possible_rules
        next_gen = []
        for i in range(0, self.possible_rules):
            if evaluate_rule[i]:
                neighborhood_state = self.possible_rules-i-1
                # print(neighborhood_state)
                neighborhood_state_bits  = self.convert_to_other_base(neighborhood_state, length = 2 * self.radius+1)
                neighborhood_states.append(neighborhood_state_bits)
                next_gen.append(evaluate_rule[i])
        # print(neighborhood_states)
        return neighborhood_states, next_gen
                
        
    def compare_state_rule(self, neighborhood, cell):
        """
        Returns whether a neighborhood is contaminated in possible_neighborhood_states()
        If this is the case return the value of a cell in the next gen
        Else return 0
        """
        rule_states, next_gen = self.possible_neighborhood_states()
        if neighborhood in rule_states:
            idx = rule_states.index(neighborhood)
            return next_gen[idx]
        else: return 0
        
        
        
    def get_neighbors(self, state):
        
        """
        Given a certain state S
        for every cell in S lookup the neighbors
        For example, given S = [1,0,0,0,0,0,0,0,1,1], base = 2 and radius = 1
        This would return the following dict for cell 8:
        8: {'left': 0, 'cell': 1, 'right': 1},
        """
        left = 0
        right = 0
        neighborhood = {}
        neighborhood_of_cell = {}
        
        for cell in range(len(state)):
            if cell == 0:
                left = 0
            else: left = state[cell-1]
            if cell == len(state)-1:
                right = 0
            else: right = state[cell+1]

            neighborhood_of_cell = {                    
            "left" : left,
            "cell" : state[cell],
            "right" : right 
            }
            neighborhood[cell] = neighborhood_of_cell
        return neighborhood
        
    
    def step(self, state):
        """
        Do a timestep in the cellular automaton
        Given a certain state perform a rule-update (transition)
        Return the new state as a list
        """
        next_state = []
        neighborhood_dict = self.get_neighbors(state)
        for cell in neighborhood_dict:
            left = neighborhood_dict[cell]["left"]
            cell_ = neighborhood_dict[cell]["cell"]
            right = neighborhood_dict[cell]["right"]
            neighbors_bits = f"{str(left)}{str(cell_)}{str(right)}"
            
            next_gen = self.compare_state_rule(neighbors_bits, cell)
            if next_gen:
                next_state.append(next_gen)
            else:
                next_state.append(0)

                    

        return next_state


    def __call__(self, c0: typing.List[int], t: int) -> typing.List[int]:
        '''Evaluate for T timesteps. Return Ct for a given C0.'''
        self.state = c0
        print(self.state)
        for i in range(t):
            print(t)
            self.state = self.step(self.state)
            print(f'generation {t}:',self.state)
        pass

In [68]:
CA = CellularAutomata(rule_number = 30, length=60, base=2)
# CA(c0=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], t = 1)
CA(c0=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],t=1)
# print(CA.possible)
# CA.get_neighbors([1,0,0,0,0,0,0,0,1,1])
# CA.rule_to_base_k()
# CA.possible_neighborhood_states()
# CA.convert_to_other_base(num=2, length=3)
# CA.convert_to_binary(num=2, length=3)
# CA.step([1,0,0,0,0,0,0,0,1,1])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1
generation 1: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## The inverse problem
* Goal: given a rule number R, a state S, and integer X, generate the initial state which will lead to S after X steps with rule R

In [322]:
lijst =  [1,2,3]
np.append(lijst,4)

array([1, 2, 3, 4])

In [469]:
def cross(parents):
        """
        * Given two parents, create a child based on a random uniform probability
        * Children are created by a simple treshold. If the probability of a certain index in idx_prob is higher than 0.5, 
        * then a gene from parent A is given, else: parent B
        * This is done for n = self.population_size 
        """
        # parentA = np.array(parents[0], dtype=int)
        # parentB = np.array(parents[1], dtype=int)
        # child = np.empty(shape=[0,1], dtype=int)
        # idx_prob = np.random.random_sample(5)
        # for i in range(len(idx_prob)):
        #     if idx_prob[i] > 0.5:
        #         child = np.append(child, parentA[i])
        #     else:
        #         child = np.append(child, parentB[i])
        # return child

        parentA = np.array(parents[0], dtype=int)
        parentB = np.array(parents[1], dtype=int)
        childA = np.empty(shape=[0,1], dtype=int)
        childB = np.empty(shape=[0,1], dtype=int)
        idx_prob = np.random.random_sample(5)
        
        for i in range(len(idx_prob)):
            if idx_prob[i] > 0.5:
                childA = np.append(childA, parentA[i])
                childB = np.append(childB, parentB[i])
            else:
                childA = np.append(childA, parentB[i])
                childB = np.append(childB, parentA[i])
        return childA, childB

def crossover(parents):
        """
        For every child in the offspring apply a crossover.
        """
        if(len(parents) != 2):
            raise ValueError("There should be 2 parents")
        offspring = []
        for i in range(10):
            offspring.append(cross(parents))

        return offspring    

    
    
p1 = [0,1,2,3,4]
p2 = [10,11,12,13,14]
p1_2 = np.empty(shape=(0, 5))
p1_2 = np.append(p1_2,np.array([p1,p2],dtype=int), axis=0)


# len(p1_2)

crossover(p1_2)

[(array([10,  1,  2, 13, 14]), array([ 0, 11, 12,  3,  4])),
 (array([10,  1,  2,  3,  4]), array([ 0, 11, 12, 13, 14])),
 (array([ 0,  1,  2, 13,  4]), array([10, 11, 12,  3, 14])),
 (array([10,  1,  2,  3,  4]), array([ 0, 11, 12, 13, 14])),
 (array([ 0,  1,  2, 13,  4]), array([10, 11, 12,  3, 14])),
 (array([10, 11, 12,  3,  4]), array([ 0,  1,  2, 13, 14])),
 (array([10, 11,  2, 13, 14]), array([ 0,  1, 12,  3,  4])),
 (array([ 0, 11,  2, 13, 14]), array([10,  1, 12,  3,  4])),
 (array([ 0, 11, 12,  3,  4]), array([10,  1,  2, 13, 14])),
 (array([10, 11, 12, 13,  4]), array([ 0,  1,  2,  3, 14]))]

In [816]:
gek = [0,2,3,4,5,6]

arr = np.empty(shape=(0,2))

arr = np.append(arr, 'pos1')
arr = np.append(arr, 'val1')
print(arr)

['pos1' 'val1']


In [833]:
len([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
)

60

In [842]:
def hamming_dist(list1, list2):
    # print('list1:',list1)
    # print('list2:',list2)
    assert len(list1) == len(list2), "The length of list1 is not equal to the length of list2"

    same_count = 0
    for i in range(len(list1)):
        if list1[i] == list2[i]:
            same_count += 1
    hamming_dist_perc = same_count/len(list1)
    return hamming_dist_perc
list1 = [0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
list2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
objective_function,hamming_dist(list1,list2)

0.3

In [22]:
import ioh
import typing
# problem = ioh.problem.wrap_integer_problem(
#         objective_function,
#         "objective_function_ca_1",
#         60, 
#         ioh.OptimizationType.Maximization,
#         ioh.IntegerConstraint([0]*60, [1]*60)
    
# )

# print(h)


def objective_function(c0_prime: typing.List[int]) -> float:
    if sum(c0_prime) > 0:
        return 1.0
    else:
        return 0.0


problem: ioh.problem.Integer = ioh.problem.wrap_integer_problem(
        objective_function,
        "objective_function",
        n_variables=5,
        optimization_type=ioh.OptimizationType.Maximization
    )
    # print('variables:', problem.meta_data.n_variables)
# ?get_problem
# print('objective.y = ', problem.objective.y)
problem([1,0,0,0,0])

1.0

In [114]:
def hamming_dist(list1, list2):
    # print(list2)
    assert len(list1) == len(list2), "The length of list1 is not equal to the length of list2"

    same_count = 0
    for i in range(len(list1)):
        if list1[i] == list2[i]:
            same_count += 1
    hamming_dist_perc = same_count/len(list1)
    return hamming_dist_perc*100

hamming_dist([1,1,1,1,1], [1,1,1,1,1])

100.0

In [116]:
ll = np.empty(shape=(3,3))

ll(0,0) = 1
ll(0,1) = 11

ll(1,0) = 2
ll(1,1) = 22
ll

SyntaxError: cannot assign to function call (1397523525.py, line 3)

In [192]:
(9**2)/(10**2)

0.81

In [194]:
10/10

1.0

In [202]:

def res():
    global groot
    groot = 2
    
res()
print(groot)

def wejow():
    groot = 
wejow()

2


UnboundLocalError: local variable 'groot' referenced before assignment

In [211]:
np.random.randint(1000)

720